In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [1]:
!kaggle competitions list

ref                                                                                deadline             category             reward  teamCount  userHasEntered  
---------------------------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
https://www.kaggle.com/competitions/ai-mathematical-olympiad-prize                 2024-06-27 23:59:00  Featured         $1,048,576        462           False  
https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability        2024-05-27 23:59:00  Featured           $105,000       3083           False  
https://www.kaggle.com/competitions/lmsys-chatbot-arena                            2024-08-05 23:59:00  Research           $100,000        157           False  
https://www.kaggle.com/competitions/learning-agency-lab-automated-essay-scoring-2  2024-07-02 23:59:00  Featured            $50,000       1214           False  
https://www.kaggle.com/competition

In [ ]:
# Download the dataset
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

In [ ]:
# Unzip the dataset
!tar -xf dogs-vs-cats-redux-kernels-edition.zip
!tar -xf test.zip
!tar -xf train.zip

**DATA EXPLORATION**

Let's have a look at the images in the folders, to get an idea of the data that we have.

In [ ]:
# Import modules
import os
from PIL import Image
import random
import matplotlib.pyplot as plt

In [ ]:
print("Number of training images: ",len(os.listdir("train")))
print("Number of testing images: ",len(os.listdir("test")))

In [ ]:
# Explore the data in the train folder

files_train = os.listdir("train")
random.shuffle(files_train)
fig = plt.figure(figsize=(15,15))
num_images = 16
for num, fn in enumerate(files_train[:num_images]):
  path = os.path.join("train",fn)
  img = Image.open(path)
  plt.subplot(num_images//4,4,num+1)
  plt.title(fn)
  plt.axis('off')
  plt.imshow(img)

In [ ]:
# Explore the data in the test folder

files_train = os.listdir("test")
random.shuffle(files_train)
fig = plt.figure(figsize=(15,15))
num_images = 16
for num, fn in enumerate(files_train[:num_images]):
  path = os.path.join("test",fn)
  img = Image.open(path)
  plt.subplot(num_images//4,4,num+1)
  plt.title(fn)
  plt.axis('off')
  plt.imshow(img)

# CODE PART 1: BUILDING AND TRAINING PLAIN-CNN

We will build and train a plain convolutional neural network without skip-connections, following the architecture described in the paper by
He et al.

Here is the schematic depiction of the network. As you can see, there are mainly 5 "blocks" of convolutional layers. Each block contains a variable number of layers, specified by a configuration list. There is one max pooling layer just after the first block, and an average pooling layer at the end before the fully connected layer. The kernel size is specified for each block, and is the same for each layer inside the block. 
Furthermore, the output channel and stride are also specified for each block.

![images_notebook/plainCNN.jpg](images_notebook/plainCNN.jpg)

Let's implement it! 

In [ ]:
from model import PlainCNN
import torch

# Testing the forward function e spiegare la formula per calcolare le dimensioni in una CNN
n_classes = 1
model = PlainCNN(config = [1,4,4,4,4], n_classes = n_classes)
model.test_forward(torch.randn(1,3,224,224))

In [ ]:
from train_utils import train
from torch.optim import SGD
from torch.nn import BCEWithLogitsLoss
import torchvision.transforms as transf
from dataset import get_datasets, get_loaders

# Define the transformations 
transforms_train = transf.Compose([
                            transf.Resize((224,224), antialias=True), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
transforms_test_val = transf.Compose([
                            transf.Resize((224,224), antialias=True), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])

# Define the datasets and dataloaders
trainset, valset, testset = get_datasets("train", "test", 0.1, 0.5, transforms_train, transforms_test_val)
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, num_workers=8, pin_memory=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model 
model = PlainCNN([1,4,4,4,4], n_classes=1)
model.to(device)
print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Define the hyperparameters 
learning_rate = 5e-3
epochs = 1

# Define the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, momentum=0.9, weight_decay=1e-4)

# Define the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory=model._getname_())

In [ ]:
import pickle

# Explore the training and validation curves 
with open(os.path.join(model._getname_(),"acc_train.pkl"),"rb") as f:
  acc_train = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_train.pkl"),"rb") as f:
  loss_train = pickle.load(f)
with open(os.path.join(model._getname_(),"acc_val.pkl"),"rb") as f:
  acc_val = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_val.pkl"),"rb") as f:
  loss_val = pickle.load(f)
  
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="Training accuracy")
plt.plot(acc_val, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="Training loss")
plt.plot(loss_val, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

### TEST THE MODEL
Now that we trained our model, we can test on some images from the test set. 

In [ ]:
from torchvision.io import read_image, ImageReadMode

# Load the model 
model = PlainCNN([1,4,4,4,4], n_classes=1)
model.load_state_dict(torch.load(os.path.join(model._getname_(),"model_1_epochs.pt")))
model.to(device)

# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test", fn)
    img = read_image(path, mode=ImageReadMode.RGB) / 255
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    img = img.permute(1,2,0)
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

# CODE PART 1: BUILDING AND TRAINING SKIP-CNN

In this part, we will build and train a "resnet" convolutional neural network **with** skip-connections, again following the architecture described in the paper by He et al.  

In [ ]:
from model import SkipCNN
import torch

# Testing the forward function e spiegare la formula per calcolare le dimensioni in una CNN
n_classes = 1
model = SkipCNN(config=[1,2,2,2,2], n_classes=1)
model.test_forward(torch.randn(1,3,224,224))

In [ ]:
from train_utils import train
from torch.optim import SGD
from torch.nn import BCEWithLogitsLoss
import torchvision.transforms as transf
from dataset import get_datasets, get_loaders

# Define the transformations 
transforms_train = transf.Compose([
                            transf.Resize((224,224), antialias=True), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
transforms_test_val = transf.Compose([
                            transf.Resize((224,224), antialias=True), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])

# Define the datasets and dataloaders
trainset, valset, testset = get_datasets("train", "test", 0.1, 0.5, transforms_train, transforms_test_val)
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, num_workers=8, pin_memory=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model 
model = SkipCNN([1,2,2,2,2], n_classes=1)
model.to(device)
print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Define the hyperparameters 
learning_rate = 5e-3
epochs = 1

# Define the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, momentum=0.9, weight_decay=1e-4)

# Define the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory=model._getname_())

In [ ]:
# Explore the training and validation curves 
with open(os.path.join(model._getname_(),"acc_train.pkl"),"rb") as f:
  acc_train = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_train.pkl"),"rb") as f:
  loss_train = pickle.load(f)
with open(os.path.join(model._getname_(),"acc_val.pkl"),"rb") as f:
  acc_val = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_val.pkl"),"rb") as f:
  loss_val = pickle.load(f)

  
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="Training accuracy")
plt.plot(acc_val, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="Training loss")
plt.plot(loss_val, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
with open(os.path.join("PlainCNN_18_layers","acc_train.pkl"),"rb") as f:
  acc_train_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","loss_train.pkl"),"rb") as f:
  loss_train_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","acc_val.pkl"),"rb") as f:
  acc_val_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","loss_val.pkl"),"rb") as f:
  loss_val_plain = pickle.load(f)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="SKIP CNN")
plt.plot(acc_train_plain, label="PLAIN CNN")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="SKIP CNN")
plt.plot(loss_train_plain, label="PLAIN CNN")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from torchvision.io import read_image, ImageReadMode

# Load the model 
model = SkipCNN([1,2,2,2,2], n_classes=1)
model.load_state_dict(torch.load(os.path.join(model._getname_(),"model_1_epochs.pt")))
model.to(device)

# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test", fn)
    img = read_image(path, mode=ImageReadMode.RGB) / 255
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    img = img.permute(1,2,0)
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

From this picture, it seems that with this shallow architectures, the skip-connections are not really necessary. Let's try with a deeper architecture, and see if the skip-connections help.

In [ ]:
from train_utils import train

# Define the datasets and dataloaders
trainset, valset, testset = get_datasets("train", "test", 0.1, 0.5, transforms_train, transforms_test_val)
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, num_workers=8, pin_memory=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model 
model = PlainCNN([1,6,8,12,6], n_classes=1) # 34 layers in total!
model.to(device)
print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Define the hyperparameters 
learning_rate = 5e-3
epochs = 1

# Define the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, momentum=0.9, weight_decay=1e-4)

# Define the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory=model._getname_())

In [ ]:
# Explore the training and validation curves 
with open(os.path.join(model._getname_(),"acc_train.pkl"),"rb") as f:
  acc_train = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_train.pkl"),"rb") as f:
  loss_train = pickle.load(f)
with open(os.path.join(model._getname_(),"acc_val.pkl"),"rb") as f:
  acc_val = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_val.pkl"),"rb") as f:
  loss_val = pickle.load(f)


plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="Training accuracy")
plt.plot(acc_val, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="Training loss")
plt.plot(loss_val, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from torchvision.io import read_image, ImageReadMode

# Load the model 
model = PlainCNN([1,6,8,12,6], n_classes=1)
model.load_state_dict(torch.load(os.path.join(model._getname_(),"model_1_epochs.pt")))
model.to(device)

# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test", fn)
    img = read_image(path, mode=ImageReadMode.RGB) / 255
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    img = img.permute(1,2,0)
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

In [ ]:
from train_utils import train

# Define the datasets and dataloaders
trainset, valset, testset = get_datasets("train", "test", 0.1, 0.5, transforms_train, transforms_test_val)
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, num_workers=8, pin_memory=True)

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model 
model = SkipCNN([1,3,4,6,3], n_classes=1)
model.to(device)
print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Define the hyperparameters 
learning_rate = 5e-3
epochs = 1

# Define the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, momentum=0.9, weight_decay=1e-4)

# Define the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory=model._getname_())

In [ ]:
# Explore the training and validation curves 
with open(os.path.join(model._getname_(),"acc_train.pkl"),"rb") as f:
  acc_train = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_train.pkl"),"rb") as f:
  loss_train = pickle.load(f)
with open(os.path.join(model._getname_(),"acc_val.pkl"),"rb") as f:
  acc_val = pickle.load(f)
with open(os.path.join(model._getname_(),"loss_val.pkl"),"rb") as f:
  loss_val = pickle.load(f)


plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="Training accuracy")
plt.plot(acc_val, label="Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="Training loss")
plt.plot(loss_val, label="Validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
with open(os.path.join("PlainCNN_18_layers","acc_train.pkl"),"rb") as f:
  acc_train_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","loss_train.pkl"),"rb") as f:
  loss_train_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","acc_val.pkl"),"rb") as f:
  acc_val_plain = pickle.load(f)
with open(os.path.join("PlainCNN_18_layers","loss_val.pkl"),"rb") as f:
  loss_val_plain = pickle.load(f)

plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(acc_train, label="SKIP CNN")
plt.plot(acc_train_plain, label="PLAIN CNN")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(loss_train, label="SKIP CNN")
plt.plot(loss_train_plain, label="PLAIN CNN")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from torchvision.io import read_image, ImageReadMode

# Load the model 
model = SkipCNN([1,3,4,6,3], n_classes=1)
model.load_state_dict(torch.load(os.path.join(model._getname_(),"model_1_epochs.pt")))
model.to(device)

# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test", fn)
    img = read_image(path, mode=ImageReadMode.RGB) / 255
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    img = img.permute(1,2,0)
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

**CODE PART 7: TRANSFER LEARNING**

In general, building a network from scratch and training it with limited data is not a good idea!
Transfer learning equip us with a strategy to leverage a pre-trained model, and adapt it to fit our task. Furthermore, pretrained networks are able to extract very general and powerful representations, which can be very useful when fine-tuned on a target dataset. 
In general, we will try to a pretrained resnet18 model, which have been trained on the ImageNet dataset (a large image classification dataset), and then fine-tune it on our cats vs dogs dataset, to see if we can achieve a better accuracy.

Usually, transfer learning is first done in those steps:
- modify the last layer of the network to fit the number of classes of the target dataset. 
- train the network by freezing all the layers in the network except the one modified.
- unfreeze the network, and train the whole network for some epochs with a lower learning rate.

Let's see how straightforward it is to do this in PyTorch.

In [ ]:
from torchvision.models import resnet18, ResNet18_Weights # Import the resnet architecture and pretrained weights from torchvision.models 

trainset, valset, testset = get_datasets("train", "test", 0.1, 1, transforms_train, transforms_test_val)
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, 8, True)

# Use cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get the model 
model = resnet18(weights=ResNet18_Weights.DEFAULT) 
# Replace the last layer to fit the output classes of the new task.
model.fc = nn.Linear(512,1)
# Freeze all layers except the last fully conneceted layer
for name, param in model.named_parameters():
  if 'fc' not in name:
    param.requires_grad = False
    
model.to(device)

print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Hyperparameters 
learning_rate = 1e-3 # With transfer learning we can use a lower learning rate
epochs = 10

# Get the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, weight_decay=1e-4, momentum=0.9) 

# Get the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory="Resnet18_linear_probing")

In [ ]:
# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.ToTensor(),
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test",fn)
    img = Image.open(path)
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

Now unfreeze the complete network and train it for 5 epochs with a lower learning rate.

In [ ]:
from torchvision.models import resnet18

trainset, valset, testset = get_datasets("train", "test", 0.1, 1, transforms_train, transforms_test_val) # Finetune on the complete dataset 
trainloader, valloader, testloader = get_loaders(trainset, valset, testset, 16, 8, True)

# Use cuda if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Get the model 
model = resnet18() 
# Replace the last layer of the model with a new one
model.fc = nn.Linear(512,1)
# Load the previous weights to continue training
model.load_state_dict(torch.load("weights/resnet18_linear_probing.pth"))

model.to(device)

print("Tot model parameters:", sum([param.numel() for param in model.parameters()]))
print("Tot trainable parameters:", sum([param.numel() for param in model.parameters() if param.requires_grad]))

# Hyperparameters 
learning_rate = 1e-4 # DECREASE THE LEARNING RATE
epochs = 5

# Get the optimizer
optimizer = SGD(model.parameters(),lr=learning_rate, weight_decay=1e-6, momentum=0.9) # DECREASE WEIGHT DECAY

# Get the loss function 
loss_fn = BCEWithLogitsLoss(reduction="mean")

# Call the train function
train(model=model, 
      trainloader=trainloader, 
      valloader=valloader, 
      epochs=epochs, 
      optimizer=optimizer,
      loss_fn=loss_fn,
      device=device,
      save_directory="Resnet18_finetuning")

In [ ]:
# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,15))
num_images = 16
transforms_test = transf.Compose([
                            transf.ToTensor(),
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test",fn)
    img = Image.open(path)
    out = model(transforms_test(img).unsqueeze(0).to(device))
    out = torch.sigmoid(out.squeeze())
    prediction = torch.round(out)
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!\n"+str(round(out.item(),2)))
    else:
      plt.title("This is a cat!\n"+str(round(out.item(),2)))

In [ ]:
# Save the weights of the networks
torch.save(model.state_dict(), "weights/resnet18_finetuning.pth")

# LET'S TRY TO PREDICT MORE IMAGES WITH OUR MOST POWERFUL MODEL! 

In [ ]:
# Explore the data 
files_test = os.listdir("test")
random.seed(0)
random.shuffle(files_test)
fig = plt.figure(figsize=(15,80))
num_images = 128
transforms_test = transf.Compose([
                            transf.ToTensor(),
                            transf.Resize((224,224)), 
                            transf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])
model.eval()
with torch.no_grad():
  for num, fn in enumerate(files_test[:num_images]):
    path = os.path.join("test",fn)
    img = Image.open(path)
    out = model(transforms_test(img).unsqueeze(0).to(device))
    prediction = out.squeeze()>0.5
    plt.subplot(num_images//4,4,num+1)
    plt.axis('off')
    plt.imshow(img)
    if(prediction):
      plt.title("This is a dog!")
    else:
      plt.title("This is a cat!")

Let's try to submit the solution to kaggle!

In [ ]:
# Submission must be in the format of a csv file with two columns: id and label
import csv

files_test = os.listdir("test")

with open('submission_kaggle.csv', 'w', newline='') as file:
  writer = csv.writer(file)

  writer.writerow(["id", "label"])
  model.eval()
  with torch.no_grad():
    for num, fn in enumerate(tqdm(files_test)): # ALL IMAGES
      id_img = fn.split(".")[0]
      path = os.path.join("test",fn)
      img = Image.open(path)
      out = model(transforms_test(img).unsqueeze(0).to(device))
      prediction = torch.clamp(out.squeeze(),min=0.02, max=0.98).item()
      writer.writerow([str(id_img),str(prediction)])
    

We would have ended up in 196 th position out of more or less 1350 people!